In [2]:
import pandas as pd

In [5]:
# import primary data
mine_pri = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-April-2024 (1).xlsx", sheet_name="Global Coal Mine Tracker (Non-C")
# import supplement data
mine_sup1 = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-September-2024-Supplement-v2 (2).xlsx", sheet_name="Historical Production (non-Chin")
mine_sup2 =pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-December-2024-Supplement-Historical-Production-from-2018-to-2023.xlsx", sheet_name="Historical Production(non-China")

In [26]:
# Step 1: Drop columns
def drop_columns(df):
    columns_to_keep = ['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)', 'Year of Production', 'Primary Consumer, Destination', 'Coal Plant, Steel Plant, Terminal', 'Owners', 'Parent Company']
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)

drop_columns(mine_pri)

,GEM Mine ID,Country,Mine Name,Owners,Parent Company,Production (Mtpa),Year of Production,Latitude,Longitude,"Primary Consumer, Destination","Coal Plant, Steel Plant, Terminal"
0,M0001,Afghanistan,Shadashak Coal Mine,North Coal Enterprise (NCE),North Coal Enterprise (NCE) [100%],1.1,NaN,35.863255,67.596674,NaN,NaN
1,M0002,Australia,Airly Coal Mine,Centennial Coal Company Pty Ltd,Banpu PCL [100.0%],1.17,2022,-33.113071,150.014937,Coal Plant,Mount Piper power station
2,M0004,Australia,Angus Place Coal Mine,Centennial Springvale Pty Ltd [50%]; Boulder M...,Banpu PCL [100.0%],*,NaN,-33.349157,150.199019,NaN,NaN
3,M0004,Australia,Angus Place Coal Mine,Centennial Springvale Pty Ltd [50%]; Boulder M...,Banpu PCL [100.0%],0,NaN,-33.351985,150.099356,NaN,NaN
4,M0005,Australia,Appin Coal Mine,Illawarra Coal,South32 Ltd [100.0%],3.9,2022,-34.211194,150.792736,Terminal (Export),Port Kembla Coal Terminal
...,...,...,...,...,...,...,...,...,...,...,...
5221,M5571,Bulgaria,Stanyantsi Mine,-,-,0.8,2016,-,-,Coal Plant,"Maritsa East Power Station, Bobov dol Power St..."
5222,M5572,China,Xinyuan UG Coal Mine,Shanxi Xinyuan Coal Company,Yang Quan Coal Industry Group,*,2022,37.912247,113.102234,NaN,NaN
5223,M5573,United States,Prairie Eagle Underground Mine,Knight Hawk Coal LLC,CBR Investments [50%]; Arch Coal [50%],3.93,2023,38.07,-89.59,NaN,NaN
5224,M5574,China,Beishan Surface Mine,Xinjiang Beishan Mining Company,Xinjiang Beishan Huiyuan Investment Company [8...,*,NaN,44.556624,90.367085,NaN,NaN


In [28]:
mine_pri = mine_pri[mine_pri['Country'] == 'India']
mine_pri.head()

,GEM Mine ID,MSHA ID,Country,Mine Name,Mine Name AKAs,Mine Name (Non-ENG),GEM Wiki URLs,Status,Status Detail,Project Type,...,Location Accuracy,"Primary Consumer, Destination","Coal Plant, Steel Plant, Terminal","Coal Plant, Steel Plant, Terminal GEM Wiki",Reported Coal Mine Methane Emissions (thousand tonnes CO2e),Year of Reported Coal Mine Methane Emissions,GEM Coal Mine Methane Emissions Estimate (MCM/yr),GEM Coal Mine Methane Emissions Estimate (M tonnes/yr),CMM Emissions (CO2e 20 years),CMM Emissions (CO2e 100 years)
482,M0464,NaN,India,Adasa Coal Mine,NaN,NaN,https://www.gem.wiki/Adasa_Coal_Mine,Operating,NaN,NaN,...,Exact,NaN,NaN,NaN,NaN,NaN,0.7,0,0,0
483,M0465,NaN,India,Adriyala Coal Mine,Adriyala Longwall Project,NaN,https://www.gem.wiki/Adriyala_coal_mine,Operating,NaN,NaN,...,Exact,Coal Plant,Ramagundam power station,https://www.gem.wiki/Ramagundam_power_station,NaN,NaN,12.6,0.008,0.7,0.3
484,M0466,NaN,India,AKK Coal Mine,NaN,NaN,https://www.gem.wiki/AKK_coal_mine,Operating,NaN,NaN,...,Exact,NaN,NaN,NaN,NaN,NaN,5.1,0.003,0.3,0.1
485,M0467,NaN,India,Amadand Coal Mine,Amadand OC,NaN,https://www.gem.wiki/Amadand_coal_mine,Operating,NaN,NaN,...,Exact,NaN,NaN,NaN,NaN,NaN,1,0.001,0.1,0
486,M0467,NaN,India,Amadand Coal Mine,NaN,NaN,https://www.gem.wiki/Amadand_coal_mine,Proposed,Permitted,Extension,...,Approximate,NaN,NaN,NaN,NaN,NaN,2.2,0.001,0.1,0


## Sheet 1: Assets

In [29]:
mine_assets = mine_pri[['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)']]
mine_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
mine_assets.head()

,asset_id,name,latitude,longitude,country,production
482,M0464,Adasa Coal Mine,21.347003,78.953545,India,0.61
483,M0465,Adriyala Coal Mine,18.6648,79.579,India,1.54
484,M0466,AKK Coal Mine,23.7905,85.9198,India,4.25
485,M0467,Amadand Coal Mine,23.157942,82.061533,India,0.87
486,M0467,Amadand Coal Mine,23.119085,81.689596,India,*


In [30]:
mine_assets['product'] = 'coal'
mine_assets.head()

/var/folders/3h/83jfd71114x78b1sj0_llf1m0000gp/T/ipykernel_9239/160352245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mine_assets['product'] = 'coal'


,asset_id,name,latitude,longitude,country,production,product
482,M0464,Adasa Coal Mine,21.347003,78.953545,India,0.61,coal
483,M0465,Adriyala Coal Mine,18.6648,79.579,India,1.54,coal
484,M0466,AKK Coal Mine,23.7905,85.9198,India,4.25,coal
485,M0467,Amadand Coal Mine,23.157942,82.061533,India,0.87,coal
486,M0467,Amadand Coal Mine,23.119085,81.689596,India,*,coal


In [31]:
prod_index = mine_assets.columns.get_loc('production')

mine_assets.insert(prod_index, 'replacement_value', '')

In [32]:
mine_assets.head()

,asset_id,name,latitude,longitude,country,replacement_value,production,product
482,M0464,Adasa Coal Mine,21.347003,78.953545,India,,0.61,coal
483,M0465,Adriyala Coal Mine,18.6648,79.579,India,,1.54,coal
484,M0466,AKK Coal Mine,23.7905,85.9198,India,,4.25,coal
485,M0467,Amadand Coal Mine,23.157942,82.061533,India,,0.87,coal
486,M0467,Amadand Coal Mine,23.119085,81.689596,India,,*,coal


## Sheet 2: Firms

In [33]:
mine_firms = mine_pri[['Owners']]
mine_firms.head()

,Owners
482,Western Coalfields Ltd
483,The Singareni Collieries Co Ltd
484,Central Coalfields Ltd
485,South Eastern Coalfields Ltd
486,South Eastern Coalfields Ltd


In [34]:
import numpy as np

In [36]:
mine_firms = mine_pri[['Owners']].rename(columns={'Owners': 'name'})

# Add required columns with empty values (NaN)
mine_firms['firm_id'] = np.nan
mine_firms['final_demand'] = np.nan
mine_firms['discount_rate'] = np.nan
mine_firms['unit_price'] = np.nan
mine_firms['margin'] = np.nan
mine_firms['leverage_ratio'] = np.nan
mine_firms['type'] = 'production'

# Reorder the columns as specified
mine_firms = mine_firms[['firm_id', 'name', 'final_demand', 'discount_rate', 'unit_price', 'margin', 'leverage_ratio', 'type']]

# Display the first few rows
mine_firms.head()

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
482,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
483,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
484,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
485,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
486,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
